# English - Alice in wonderland

## Data preparation

In [1]:
from importlib import reload

import preprocessor
preprocessor = reload(preprocessor)


pp = preprocessor.Preprocessor("data/original/alice_in_wonderland.txt", "eng")
cleaned_corpus = pp.process()
pp.split(cleaned_corpus)

## Subword segmentation

In [5]:
import sentpiece
sentpiece = reload(sentpiece)


NUM_CHARS = 44
TRAIN_DATA_PATH = "data/processed/eng_train.txt"
MODEL_NAME = "eng_model_chr"
SEG_DATA_PATH = "eng_chr.txt"


sentpiece.train_model(TRAIN_DATA_PATH, MODEL_NAME, NUM_CHARS)

In [6]:
!mv $MODEL_NAME* spm_models/

In [7]:
sentpiece.segmentation(TRAIN_DATA_PATH, MODEL_NAME, SEG_DATA_PATH)

## Train LM

In [14]:
TRAIN_SEG_DATA_PATH = "data/segmented/eng_chr.txt"
NUM_LAYERS = 40
CLASS = 9999
BPTT = 3


!bash train_script.sh $TRAIN_SEG_DATA_PATH "data/processed/eng_test.txt" $NUM_LAYERS $CLASS $BPTT

rm: cannot remove 'rnnlm/models/model': No such file or directory
rm: cannot remove 'rnnlm/models/model.output.txt': No such file or directory
debug mode: 2
train file: data/segmented/eng_chr.txt
valid file: data/processed/eng_test.txt
class size: 9999
Hidden layer size: 40
BPTT: 3
Rand seed: 1
rnnlm file: model
Starting training using file data/segmented/eng_chr.txt
Vocab size: 42
Words in train file: 113695
Iter:   0	Alpha: 0.100000	   TRAIN entropy: 3.0615    Words/sec: 571.7   VALID entropy: 12.5994
Iter:   1	Alpha: 0.100000	   TRAIN entropy: 2.6278    Words/sec: 511.4   VALID entropy: 13.6886
Iter:   2	Alpha: 0.050000	   TRAIN entropy: 2.5822    Words/sec: 493.1   VALID entropy: 13.6706

real	11m4.928s
user	11m2.247s
sys	0m0.222s


In [15]:
MODEL_OUTPUT = MODEL_NAME + ".output.txt"

!mv model $MODEL_NAME
!mv model.output.txt $MODEL_OUTPUT

!mv $MODEL_NAME $MODEL_OUTPUT rnnlm_models/

mv: target 'rnnlm_models/' is not a directory


## Data generation

In [16]:
GEN_DATA_PATH = "data/generated/eng_model_chr/"
MODEL_PATH = f"./rnnlm_models/{MODEL_NAME}"

!bash gen_script.sh $GEN_DATA_PATH $MODEL_PATH

# Bengali

# Analysis